## 단계별 작업

In [2]:
import random
import pandas as pd
import os
pd.options.display.max_rows=20

In [4]:
folder = '201706'
os.chdir('/home/yahwang/Desktop/bikeseoul/census/일별(시간)/'+ folder)

In [5]:
os.path.join(os.getcwd(), os.listdir()[0])

'/home/yahwang/Desktop/bikeseoul/census/일별(시간)/201706/LOCAL_PEOPLE_20170621.csv'

In [6]:
df = pd.read_csv(os.path.join(os.getcwd(), os.listdir()[0]))
df.head()

,기준일ID,시간대구분,행정동코드,집계구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20170621,0,11110515,1101072010001,716,23,15,22,36,29,...,20,22,21,41,31,32,25,20,7,41
1,20170621,0,11110515,1101072010002,497,17,11,13,13,12,...,17,20,24,26,24,16,16,14,13,43
2,20170621,0,11110515,1101072010003,410,23,15,14,13,12,...,9,5,16,15,27,14,9,12,11,63
3,20170621,0,11110515,1101072010004,263,13,8,7,13,8,...,8,10,15,6,12,9,6,12,*,8
4,20170621,0,11110515,1101072010005,561,28,18,18,21,19,...,14,10,20,25,35,20,14,16,12,67


####  기본 전처리

In [7]:
# 인구가 3이하인 데이터를 *로 표시한 문제 -> 0-3을 random값으로 주고 int 타입으로 변경
df.iloc[:,5:] = df.iloc[:,5:].replace('*', random.randint(0,3))
df.iloc[:,5:] = df.iloc[:,5:].astype('int')

df.iloc[:,1] = pd.to_timedelta(df.iloc[:,1], unit='h') # 시간 계산을 위한 타입 변경
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], format='%Y%m%d') # 20170707 int데이터를 datetime 데이터로 변경
df.iloc[:,0] = df.iloc[:,0]+df.iloc[:,1] # 시간을 추가한 datetime 타입으로 합침

df.iloc[:,2] = df.iloc[:,2]*100 # 행정코드에 00 추가
# 필요없는 컬럼 제거
df.drop(columns=['집계구코드','시간대구분','총생활인구수'], inplace=True)

#### 나이대로 결합

In [79]:
# 나이대로 컬럼 합치는 함수
def concat_col(df, col_num1, col_num2):
    df.iloc[:,col_num1] += df.iloc[:,col_num2]

In [80]:
# 컬럼의 위치 index를 사용해서 전처리
concat_col(df,3,4) # 남 10대
concat_col(df,5,6) # 남 20대
concat_col(df,7,8) # 남 30대
concat_col(df,9,10) # 남 40대
concat_col(df,11,12) # 남 50대
concat_col(df,13,14) # 남 60대

concat_col(df,17,18) # 여 10대
concat_col(df,19,20) # 여 20대
concat_col(df,21,22) # 여 30대
concat_col(df,23,24) # 여 40대
concat_col(df,25,26) # 여 50대
concat_col(df,27,28) # 여 60대
del_col_num = [2,4,6,8,10,12,14,16,18,20,22,24,26,28]
df.drop(columns=df.columns[del_col_num], inplace=True)

In [81]:
df.columns = ['date','area','m_age_10','m_age_20','m_age_30','m_age_40','m_age_50','m_age_60','m_age_70','f_age_10','f_age_20','f_age_30','f_age_40','f_age_50','f_age_60','f_age_70'] 
df = df.groupby(['date','area']).agg('sum') # 집계구 단위 데이터를 행정동 단위로 묶는다.
df.reset_index(inplace=True)

In [82]:
df.head()

,date,area,m_age_10,m_age_20,m_age_30,m_age_40,m_age_50,m_age_60,m_age_70,f_age_10,f_age_20,f_age_30,f_age_40,f_age_50,f_age_60,f_age_70
0,2017-07-04,1111051500,807,1068,1202,1320,951,508,874,658,897,1170,1591,1041,810,1437
1,2017-07-04,1111053000,446,1477,1772,1950,1449,1005,1168,546,1355,1729,1673,1378,1161,1904
2,2017-07-04,1111054000,377,439,327,392,297,293,263,190,234,395,398,347,312,507
3,2017-07-04,1111055000,590,853,702,894,920,669,814,680,837,837,1173,1004,983,1169
4,2017-07-04,1111056000,1137,1335,1171,1604,1834,1330,1456,980,1376,1453,1875,2109,1718,2673


#### 30분 단위로 만들기 ( 1시간 단위는 불필요 )
    30분 단위로 만들기 위해 date(30분단위), area를 기반으로 새로운 index를 먼저 만들고 JOIN한다.

In [86]:
# df_new_idx : 새로 만든 index를 가진 데이터프레임
now = os.listdir()[0][13:-4]
minute30= pd.date_range(now, now+' 23:30:00', freq='30T') # 30분 단위 index 생성
new_index =pd.MultiIndex.from_product([minute30, df['area'].unique()]) # 30분 index와 동코드를 결합 (48 * 424)
df_new_idx = pd.DataFrame(index=new_index) # 새 index로 데이터프레임 생성 
df_new_idx.reset_index(inplace=True)
df_new_idx.columns = ['date','area']

df = pd.merge(df_new_idx, df, how='left')  # 데이터를 조인해서 30분단위를 null값으로 채움
df.sort_values(by=['area','date'], inplace=True) # 빈 값을 앞뒤 평균으로 채우기 위해 동별 시간 단위로 정렬

In [87]:
df.head()

,date,area,m_age_10,m_age_20,m_age_30,m_age_40,m_age_50,m_age_60,m_age_70,f_age_10,f_age_20,f_age_30,f_age_40,f_age_50,f_age_60,f_age_70
0,2017-07-04 00:00:00,1111051500,807.0,1068.0,1202.0,1320.0,951.0,508.0,874.0,658.0,897.0,1170.0,1591.0,1041.0,810.0,1437.0
424,2017-07-04 00:30:00,1111051500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848,2017-07-04 01:00:00,1111051500,826.0,1079.0,1182.0,1330.0,979.0,486.0,915.0,656.0,903.0,1179.0,1593.0,1050.0,818.0,1446.0
1272,2017-07-04 01:30:00,1111051500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1696,2017-07-04 02:00:00,1111051500,816.0,1099.0,1202.0,1348.0,990.0,481.0,916.0,664.0,890.0,1162.0,1593.0,1055.0,824.0,1459.0


#### 동 코드를 동이름으로 변환

In [88]:
area_code = pd.read_csv('/home/yahwang/Downloads/유동인구/areacode.csv')
area_code.drop(columns=['행정구역분류'],inplace=True)
area_code.columns = ['gu','dong','area']
area_code.head()

,gu,dong,area
0,종로구,청운효자동,1111051500
1,종로구,사직동,1111053000
2,종로구,삼청동,1111054000
3,종로구,부암동,1111055000
4,종로구,평창동,1111056000


In [89]:
df = pd.merge(df, area_code)
df = df.reindex(['date','gu','dong']+df.columns[1:-2].tolist(), axis=1) # 구, 동 컬럼을 앞으로 배치

In [90]:
df.head()

,date,gu,dong,area,m_age_10,m_age_20,m_age_30,m_age_40,m_age_50,m_age_60,m_age_70,f_age_10,f_age_20,f_age_30,f_age_40,f_age_50,f_age_60,f_age_70
0,2017-07-04 00:00:00,종로구,청운효자동,1111051500,807.0,1068.0,1202.0,1320.0,951.0,508.0,874.0,658.0,897.0,1170.0,1591.0,1041.0,810.0,1437.0
1,2017-07-04 00:30:00,종로구,청운효자동,1111051500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-04 01:00:00,종로구,청운효자동,1111051500,826.0,1079.0,1182.0,1330.0,979.0,486.0,915.0,656.0,903.0,1179.0,1593.0,1050.0,818.0,1446.0
3,2017-07-04 01:30:00,종로구,청운효자동,1111051500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-04 02:00:00,종로구,청운효자동,1111051500,816.0,1099.0,1202.0,1348.0,990.0,481.0,916.0,664.0,890.0,1162.0,1593.0,1055.0,824.0,1459.0


## 한 폴더 한 번에 작업하기
    - 201707 폴더에 있는 모든 일별 파일들을 함수를 통해 폴더명으로만 실행한다.

In [28]:
import random
import pandas as pd
import os
from tqdm import tqdm
pd.options.display.max_rows=20

def preprocess(folder):
    save_path = '/home/yahwang/Desktop/bikeseoul/census/'+folder # 저장 폴더 생성
    if not os.path.exists(save_path): 
        os.mkdir(save_path)

    # 동 코드 정리된 파일
    area_code = pd.read_csv('/home/yahwang/Desktop/bikeseoul/census/areacode.csv')
    area_code.drop(columns=['행정구역분류'],inplace=True)
    area_code.columns = ['gu','dong','area']

    os.chdir('/home/yahwang/Desktop/bikeseoul/census/LOCAL_PEOPLE_'+ folder) # 작업할 폴더 위치 설정

    # 나이대로 컬럼 합치는 함수
    def concat_col(df, col_num1, col_num2):
        df.iloc[:,col_num1] += df.iloc[:,col_num2]
            
    for file in tqdm(os.listdir()):
        df = pd.read_csv(os.path.join(os.getcwd(), file))

        ####  기본 전처리

        # 인구가 3이하인 데이터를 *로 표시한 문제 -> 0-3을 random값으로 주고 int 타입으로 변경
        df.iloc[:,5:] = df.iloc[:,5:].replace('*', random.randint(0,3))
        df.iloc[:,5:] = df.iloc[:,5:].astype('int')

        df.iloc[:,1] = pd.to_timedelta(df.iloc[:,1], unit='h') # 시간 계산을 위한 타입 변경
        df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], format='%Y%m%d') # 20170707 int데이터를 datetime 데이터로 변경
        df.iloc[:,0] = df.iloc[:,0]+df.iloc[:,1] # 시간을 추가한 datetime 타입으로 합침
        df.iloc[:,2] = df.iloc[:,2]*100 # 행정코드에 00 추가
        
        # 필요없는 컬럼 제거
        df.drop(columns=['집계구코드','시간대구분','총생활인구수'], inplace=True)

        #### 나이대로 결합

        concat_col(df,3,4) # 남 10대
        concat_col(df,5,6) # 남 20대
        concat_col(df,7,8) # 남 30대
        concat_col(df,9,10) # 남 40대
        concat_col(df,11,12) # 남 50대
        concat_col(df,13,14) # 남 60대

        concat_col(df,17,18) # 여 10대
        concat_col(df,19,20) # 여 20대
        concat_col(df,21,22) # 여 30대
        concat_col(df,23,24) # 여 40대
        concat_col(df,25,26) # 여 50대
        concat_col(df,27,28) # 여 60대
        
        del_col_num = [2,4,6,8,10,12,14,16,18,20,22,24,26,28]
        df.drop(columns=df.columns[del_col_num], inplace=True) # 합친 후 필요없는 컬럼 제거

        ### 행정동에 맞게 처리

        df.columns = ['date','area','m_age_10','m_age_20','m_age_30','m_age_40','m_age_50','m_age_60','m_age_70','f_age_10','f_age_20','f_age_30','f_age_40','f_age_50','f_age_60','f_age_70'] 
        df = df.groupby(['date','area']).agg('sum') # 집계구 단위 데이터를 행정동 단위로 묶는다.
        df.reset_index(inplace=True)
        
        ### 30분 단위 데이터 만들기
        
        # df_new_idx : 새로 만든 index를 가진 데이터프레임
#         now = file[13:-4] # 파일 이름에서 날짜만 추출
#         minute30= pd.date_range(now, now+' 23:30:00', freq='30T') # 30분 단위 index 생성
#         new_index =pd.MultiIndex.from_product([minute30, df['area'].unique()]) # 30분 index와 동코드를 결합 (48 * 424)
#         df_new_idx = pd.DataFrame(index=new_index) # 새 index로 데이터프레임 생성
#         df_new_idx.reset_index(inplace=True)
#         df_new_idx.columns = ['date','area']

#         df = pd.merge(df_new_idx, df, how='left') # 데이터를 조인해서 30분단위를 null값으로 채움
#         df.sort_values(by=['area','date'], inplace=True) # 빈 값을 앞뒤 평균으로 채우기 위해 동별 시간 단위로 정렬

        ### 1시간 단위는 30분 만들기 건너띄고 여기부터 시작
        
        df = pd.merge(df, area_code)     #### 동 코드를 동이름으로 변환
        df = df.reindex(['date','gu','dong']+df.columns[1:-2].tolist(), axis=1) # 구, 동 컬럼을 앞으로 배치
        df.to_csv(os.path.join(save_path, file), index=False) # csv 파일로 추출

In [47]:
%time preprocess('201803') # 테스트


100%|██████████| 31/31 [02:33<00:00,  4.92s/it]


CPU times: user 2min 3s, sys: 30.5 s, total: 2min 34s
Wall time: 2min 33s


In [41]:
# 폴더명들을 리스트로 생성
folder_list = list(map(lambda x: x.strftime('%Y%m'), pd.date_range('20170901','20180630',freq='M').tolist()))
folder_list

['201709',
 '201710',
 '201711',
 '201712',
 '201801',
 '201802',
 '201803',
 '201804',
 '201805',
 '201806']

In [ ]:
# 함수실행
%time for folder in tqdm(folder_list): preprocess(folder)

## 일 단위 파일 하나로 합치기

In [4]:
import pandas as pd
import os

folder = '201706'
os.chdir('/home/yahwang/Desktop/bikeseoul/census/'+folder)

In [5]:
total_df = pd.read_csv(os.path.join(os.getcwd(),os.listdir()[0]))
for file in os.listdir()[1:]:
    total_df = pd.concat([total_df, pd.read_csv(os.path.join(os.getcwd(),file))], ignore_index=True)

In [6]:
total_df.shape

(305280, 18)

In [7]:
total_df.sort_values(by=['area','date'], inplace=True)
total_df.reset_index(drop=True,inplace=True)

In [9]:
total_df.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+folder+'.csv',index=False)

### 테스트 : 하나의 월로 30분 단위일 때 NULL 평균값으로 채우기

In [23]:
total_df = total_df.groupby(['area']).apply(lambda group: group.interpolate()) # 평균값으로 채우기

In [24]:
total_df.iloc[1482:1495] # 제대로 되는 지 확인 ( 그룹 내에서만 계산하는가 )

,date,gu,dong,area,m_age_10,m_age_20,m_age_30,m_age_40,m_age_50,m_age_60,m_age_70,f_age_10,f_age_20,f_age_30,f_age_40,f_age_50,f_age_60,f_age_70
1482,2017-06-02 18:00:00,종로구,삼청동,1111054000,459,788,1149,1086,652,350,300,343,943,910,730,672,420,461
1483,2017-06-02 19:00:00,종로구,삼청동,1111054000,456,661,928,882,647,330,352,285,909,809,694,620,399,523
1484,2017-06-02 20:00:00,종로구,삼청동,1111054000,404,579,824,745,590,323,270,261,780,695,581,519,371,515
1485,2017-06-02 21:00:00,종로구,삼청동,1111054000,369,524,685,609,477,279,237,266,563,569,461,451,341,509
1486,2017-06-02 22:00:00,종로구,삼청동,1111054000,219,453,560,541,397,283,245,194,436,477,373,419,307,493
1487,2017-06-02 23:00:00,종로구,삼청동,1111054000,157,445,555,546,396,282,239,182,355,431,349,412,361,510
1488,2017-06-03 00:00:00,종로구,삼청동,1111054000,159,419,491,475,359,268,241,186,324,420,339,381,339,516
1489,2017-06-03 01:00:00,종로구,삼청동,1111054000,170,452,503,476,368,273,243,182,328,422,342,398,343,534
1490,2017-06-03 02:00:00,종로구,삼청동,1111054000,177,461,494,496,362,263,262,179,341,441,348,406,348,565
1491,2017-06-03 03:00:00,종로구,삼청동,1111054000,190,450,499,489,367,283,246,179,345,447,360,417,345,552


## 모든 월 한번에 합쳐서 평균값으로 채우기 (30분 단위 일 때)

### 1. Null값 그대로 두고 월별 합산

In [48]:
import pandas as pd
import os
folder_list = list(map(lambda x: x.strftime('%Y%m'), pd.date_range('20180301','20180630',freq='M').tolist()))

In [49]:
for folder in folder_list:
    os.chdir('/home/yahwang/Desktop/bikeseoul/census/일별(시간)/'+folder)
    total_df = pd.read_csv(os.listdir()[0])
    for file in os.listdir()[1:]:
        total_df = pd.concat([total_df, pd.read_csv(file)], ignore_index=True)
    
    # 행정동, 일 별로 다시 정리 ( NULL 값이 사이사이 들어가야 하는 이슈 때문 )
    total_df.sort_values(by=['area','date'], inplace=True)
    total_df.reset_index(drop=True,inplace=True)
    total_df.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+folder+'.csv',index=False)

### 2. 모든 월을 한 파일로 통합 후 NULL 값 채우기

In [1]:
import pandas as pd
import os

In [2]:
os.chdir('/home/yahwang/Desktop/bikeseoul/census/월별(시간)/')
df = pd.read_csv(os.listdir()[0])
for file in os.listdir()[1:]:
    df = pd.concat([df,pd.read_csv(file)])

# 행정동, 일 별로 다시 정리 ( NULL 값이 사이사이 들어가야 하는 이슈 때문 )
## 월 별에서 실행하지 안고 여기서 한 번에 해도 되는 듯
df.sort_values(by=['area','date'], inplace=True) 
df.reset_index(drop=True,inplace=True)

# df = df.groupby(['area']).apply(lambda group: group.interpolate()) # 평균값으로 채우기 ( 메모리 부족으로 colab에서 하는 걸 추천 )
df.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+'census_train.csv',index=False)

### Z-표준화

In [10]:
import numpy as np
import pandas as pd
df2 = pd.read_csv('/home/yahwang/Desktop/bikeseoul/census/zscore/census_train.csv')

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714240 entries, 0 to 3714239
Data columns (total 18 columns):
date        object
gu          object
dong        object
area        int64
m_age_10    int64
m_age_20    int64
m_age_30    int64
m_age_40    int64
m_age_50    int64
m_age_60    int64
m_age_70    int64
f_age_10    int64
f_age_20    int64
f_age_30    int64
f_age_40    int64
f_age_50    int64
f_age_60    int64
f_age_70    int64
dtypes: int64(15), object(3)
memory usage: 510.1+ MB


#### train standardization의 사용한 결과를 저장
    - test 데이터에도 똑같은 기준으로 적용해야 한다.

In [34]:
avg_val = df2.iloc[:,4:].apply('mean')
avg_val = avg_val.to_frame()
avg_val.columns = ['mean']

std_val = df2.iloc[:,4:].apply('std')
std_val = std_val.to_frame()
std_val.columns = ['std']

train_val = pd.concat([avg_val,std_val], axis=1, sort=False)
train_val.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+'calc_val.csv')

In [7]:
# apply로 한번에 컬럼별 계산
df2.iloc[:,4:] = df2.iloc[:,4:].apply(lambda x: (x-np.mean(x))/np.std(x))

In [8]:
df2.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+'census_train(zscore).csv',index=False)

test 데이터용

In [31]:
import pandas as pd

In [32]:
df3 = pd.read_csv('/home/yahwang/Desktop/bikeseoul/census/zscore/census_test.csv')

In [33]:
train_val = pd.read_csv('/home/yahwang/Desktop/bikeseoul/census/calc_val.csv')

In [34]:
i = 0
for index in range(4,18):
    _,mean,std = train_val.iloc[i]
    df3.iloc[:,index] = df3.iloc[:,4:].apply(lambda x: (x-mean)/std)
    print(mean, std)
    i+=1

1040.2489559102266 621.0755706468004
1930.6675761393985 1774.527880073605
2097.987232919789 1975.9502187050791
2021.8296464418024 1676.5467567386027
1682.5796631343155 1014.9667002940737
1252.8037022378735 640.827958462186
1126.7791669359008 582.2292068690716
1033.0576325170157 685.1250046911329
2121.4076543249757 2177.1373649534794
2087.750102308952 1604.6130550706926
1948.3617706448695 1130.8739027309475
1889.1212223765829 913.2431918516517
1581.5429145666405 733.9768069813606
1791.3073433596105 829.166926087997


In [35]:
df3.head()

,date,gu,dong,area,m_age_10,m_age_20,m_age_30,m_age_40,m_age_50,m_age_60,m_age_70,f_age_10,f_age_20,f_age_30,f_age_40,f_age_50,f_age_60,f_age_70
0,2018-06-01 00:00:00,종로구,청운효자동,1111051500,-0.734611,-0.513752,-0.512658,-0.404301,-0.704042,-1.131043,-0.696941,-0.581000,-0.591330,-0.543277,-0.365524,-0.871752,-1.007583,-0.508109
1,2018-06-01 01:00:00,종로구,청운효자동,1111051500,-0.774864,-0.503609,-0.524298,-0.412652,-0.698131,-1.134164,-0.707246,-0.573702,-0.593168,-0.554495,-0.354913,-0.869562,-1.030745,-0.537054
2,2018-06-01 02:00:00,종로구,청운효자동,1111051500,-0.758763,-0.517697,-0.522780,-0.424581,-0.695175,-1.113877,-0.714116,-0.570783,-0.588575,-0.544524,-0.372598,-0.875037,-1.044369,-0.502079
3,2018-06-01 03:00:00,종로구,청운효자동,1111051500,-0.760373,-0.521642,-0.529359,-0.446650,-0.719807,-1.146647,-0.693506,-0.594136,-0.592249,-0.569452,-0.383210,-0.927597,-1.067531,-0.474340
4,2018-06-01 04:00:00,종로구,청운효자동,1111051500,-0.761983,-0.523896,-0.526829,-0.444861,-0.758231,-1.177857,-0.750184,-0.613111,-0.599139,-0.583786,-0.425655,-0.941832,-1.063444,-0.528612


In [36]:
df3.to_csv('/home/yahwang/Desktop/bikeseoul/census/'+'census_test(zscore).csv', index=False)